In [212]:
import pandas as pd
from datetime import datetime
import os,random

* ### Извлечение данных

In [213]:
def extract_data(path): 
    """
    Извлекает все данные из папки, по данному пути.
    :param path: - путь
    """
    files = os.listdir(path); csv_files = [file for file in files if file.endswith('.csv')]
    dataframes = {}
    
    for file in csv_files:
        file_path = os.path.join(path, file)
        df = pd.read_csv(file_path, sep = '\t')
        df['timestamp'] = df['timestamp'].apply(lambda x: pd.Timestamp(x).to_pydatetime())
        dataframes[file] = df

    return dataframes

In [214]:
all_data = extract_data('/home/milica/тестовое вк/data_for_testing')
data = all_data['variant1.csv']
display(data.sort_values('timestamp'))

,userid,timestamp,action,value,testids
5062,user_475,2021-04-01 01:58:37,confirmation,94394,7
3778,user_31,2021-04-01 03:57:17,checkout,50949,25;54;60;73;12;30;2;36
8546,user_103,2021-04-01 06:23:22,cart,42398,84;78;80;31;52
2950,user_46,2021-04-01 07:09:04,mainpage,0,84;79;3;86;28;38;48
4626,user_313,2021-04-01 18:44:49,mainpage,0,27;40;85;12;5;96;30;38
...,...,...,...,...,...
4749,user_306,2024-03-31 10:01:24,category,0,77;21;57;54;19;8
8687,user_46,2024-03-31 13:27:40,mainpage,0,67;58
3510,user_360,2024-03-31 18:25:20,confirmation,81923,32;76;51;40;66;19
7624,user_425,2024-03-31 18:53:19,cart,62450,18;40;56;24


In [215]:
display(data[data['action'] == 'confirmation'].sort_values('timestamp'))

,userid,timestamp,action,value,testids
5062,user_475,2021-04-01 01:58:37,confirmation,94394,7
447,user_20,2021-04-04 06:11:34,confirmation,90169,NaN
8595,user_227,2021-04-04 23:00:51,confirmation,40744,67;81
8109,user_324,2021-04-05 00:00:36,confirmation,65842,9;36;81;95;14;49;26;29;59
567,user_129,2021-04-05 12:22:22,confirmation,18836,22;86
...,...,...,...,...,...
138,user_149,2024-03-27 17:13:28,confirmation,49546,94;93;31;67;26
6207,user_351,2024-03-27 17:46:46,confirmation,81892,55;16;76;40;59;15;94;56;3
4476,user_68,2024-03-29 05:34:45,confirmation,38593,2;60;17;96;81;81;95;78;85
745,user_436,2024-03-29 09:37:00,confirmation,47152,80;10;8;4;30


* ### Обработка данных

In [216]:
def filter_months_duplicates(df):
    """
    Сортирует таблицу по возрастанию месяцев, в случае нескольких максимальных покупок в месяц выбирает первую из них.
    :param df: - data frame
    """
    df['timestamp'] = df['timestamp'].apply(lambda x: pd.Timestamp(x).to_pydatetime())
    df['year_month'] = df['timestamp'].dt.to_period('M')
    max_by_month = df.groupby('year_month')['value'].transform('max')
    df_max = df[df['value'] == max_by_month]
    df_max_sorted = df_max.sort_values('timestamp').drop_duplicates('year_month',keep="first")

    return df_max_sorted[['timestamp','value']]

In [217]:
def extract_full_months(df):
    """
    Находит лучший день для каждого полноценного месяца использования сайта. Возращает таблицу с колонками: timestamp, value.
    :param df: - data frame
    """
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date # оставляем только даты, без часов
    users_per_month = df.groupby(['timestamp'])['userid'].unique().reset_index() # группируем пользователей по датам
    full_months = pd.DataFrame(columns=['timestamp','value'])
    min_date, max_date = df['timestamp'].agg(['min', 'max'])
    
    for index, row in users_per_month.iterrows():
        year = row['timestamp'].year; month = row['timestamp'].month
        #проверка на то, является ли месяц первым или последним месяцем, для которого представлены данные
        if (year == min_date.year and month == min_date.month) or (year  == max_date.year and month == max_date.month):
            continue 
            
        unique_users = row['userid']
        #отделяем всех пользователей, которые уже были в таблице  
        old_users = df[(df['timestamp'] < row['timestamp'])]['userid'] 
        #отделяем новых пользователей, пришедших в эту дату
        new_users =  set(unique_users).difference(set(old_users))  
        #находим новых пользователей, совершивших покупки в эту дату
        users_with_conformation = df[(df['userid'].isin(new_users)) & (df['action'] == 'confirmation') & (df['timestamp'] == row['timestamp'])]
        if not users_with_conformation.empty:
                max_user = users_with_conformation.loc[users_with_conformation['value'].idxmax()]
                new_row = {'timestamp': max_user['timestamp'],  'value': max_user['value']}
                full_months.loc[len(full_months)] = new_row
                full_months = full_months.reset_index(drop=True)
     
    full_months = filter_months_duplicates(full_months) 
    
    return full_months

In [218]:
data1 = extract_full_months(data)
display(data1)

,timestamp,value
3,2021-05-08,96806
12,2021-06-05,58894
22,2021-07-28,99161
23,2021-08-02,91481
25,2021-09-27,99243
26,2021-10-01,77590
29,2022-01-01,93900


* ### Тест на 10 случайных выборок

In [219]:
def test_random_data(n,data):
    for i in range(n):
        key = random.choice(list(data.keys()))
        print("Файл =", key)
        display(extract_full_months(data[key]))

In [220]:
test_random_data(10,all_data)

Файл = variant91.csv


,timestamp,value
6,2021-05-19,95005
13,2021-06-13,93061
18,2021-07-23,94595
21,2021-08-07,81835
24,2021-09-15,91677
27,2021-10-27,56941


Файл = variant85.csv


,timestamp,value
0,2021-05-01,94657
19,2021-06-27,92683
24,2021-07-18,74408
32,2021-08-27,92817
34,2021-09-09,91314
36,2021-10-08,98478
38,2021-11-04,20882


Файл = variant9.csv


...

,timestamp,value
5,2021-05-12,85695
19,2021-06-11,97394
26,2021-07-12,92879
34,2021-08-19,62252
36,2021-09-20,65806
38,2021-10-27,97138
39,2021-11-08,75954
40,2022-04-16,57141


Файл = variant64.csv


,timestamp,value
8,2021-05-11,93859
21,2021-06-10,99478
31,2021-07-18,88804
40,2021-08-29,92705
41,2021-09-10,23169
43,2021-10-20,38822
44,2021-11-12,6827


Файл = variant22.csv


,timestamp,value
12,2021-05-29,99954
16,2021-06-14,87451
25,2021-07-15,90242
31,2021-08-13,43509
32,2021-09-01,87128
